# iCard Data Comparison (Sampled vs Filtered)

In [1]:
import psycopg2
import pandas as pd
import config
from matplotlib import pylab as plt
import seaborn as sns
import numpy as np
from IPython.display import display, HTML
import re


color_palette = sns.color_palette(palette='muted', n_colors=None, desat=.75)
sns.set(context='notebook', palette=color_palette, style='whitegrid', font='sans-serif', font_scale=1.5, color_codes=False, rc=None)
pd.set_option('display.max_colwidth', -1)
table_styles = [{'selector': 'td',
                 'props': [('min-width', '100px'), ('text-align', 'center')]},
                {'selector': 'tr',
                 'props': [('border-bottom', '1px dotted black')]},
                {'selector': 'th',
                 'props': [('text-align', 'center')]}
               ]

%matplotlib inline

directory = "url_top_lists/"
stream = "comparison"

/media/raid/home/pflugmacher/virtualenv/twitterenvold/lib/python3.4/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/media/raid/home/pflugmacher/virtualenv/twitterenvold/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.DataFrame(data={'tags':['a', 'b', 'c', 'd', 'a', 'b']})
list(df[df['tags']=='a'].tags.values)

['a', 'a']

## Util Methods

In [3]:
def compareRows(row, df_to_compare, column_name):
    comparison_row = df_to_compare.loc[df_to_compare[column_name] == row[column_name]]
    if comparison_row.empty:
        comparison = " - "
    else:
        percentage_dif = row['percentage'] - comparison_row['percentage'].values[0]
        difference = "(%s. / %.3f%% / %.3f%%)" % (comparison_row['rank'].values[0], comparison_row['percentage'].values[0], percentage_dif)
        if comparison_row['rank'].values[0] == row['rank']:
            comparison = " = <br>" + difference
        else:
            if comparison_row['rank'].values[0] > row['rank']:
                comparison = " v <br>" + difference
            else:
                comparison = " ^ <br>" + difference
    return comparison

def getOpacity(val):
    value = abs(float(re.findall(r"[-+]?\d*\.\d+|\d+", val.split("/")[2])[0]))
    if value < 0.005:
        return 1
    if value < 0.01:
        return 0.95
    if value < 0.05:
        return 0.8
    if value < 0.1:
        return 0.7
    if value < 0.5:
        return 0.6
    if value < 1:
        return 0.5
    if value < 10:
        return 0.3
    if value < 40:
        return 0.2
    if value < 80:
        return 0.1
    if value < 100:
        return 0.05

def colorComparisonField(val):
    
    if isinstance(val, str):
        if ' ^ ' in val or ' v ' in val:
            return 'background-color: rgba(246, 185, 59, %s)' %getOpacity(val)
        if ' = ' in val:
            return 'background-color: rgba(184, 233, 148, %s)' %getOpacity(val)
        if ' - ' in val and len(val) == 3:
            return 'background-color: #e55039' 
    return ''

def generateRankingDataframe(series, attribute_name):
    size = series.sum()
    rank = []
    parameter = []
    count = []
    percentage = []

    i = 1
    for index, value in series.iteritems():
        rank.append(i)
        parameter.append(index)
        count.append(value)
        percentage.append((value/size)*100)
        i += 1

    data = {'rank': rank, attribute_name: parameter, 'value': count, 'percentage': percentage}
    return pd.DataFrame(data=data)

def generateComparisonDataframes(df1, df2, column_name, size):
    compare_list = []
    for index, row in df1.iterrows():
        if row['rank'] <= size:
            compare_list.append(compareRows(row, df2, column_name))

    data = {'rank': df1['rank'][:size], column_name: df1[column_name][:size], 'value': df1['value'][:size], 'percentage': df1['percentage'][:size],
            'difference (rank / percentage / diff)': compare_list}
    
    df1_compared = pd.DataFrame(data=data)
    df1_compared.set_index(keys='rank', inplace=True)
    
    compare_list = []
    for index, row in df2.iterrows():
        if row['rank'] <= size:
            compare_list.append(compareRows(row, df1, column_name))

    data = {'rank': df2['rank'][:size], column_name: df2[column_name][:size], 'value': df2['value'][:size], 'percentage': df2['percentage'][:size],
            'difference (rank / percentage / diff)': compare_list}
    
    df2_compared = pd.DataFrame(data=data)
    df2_compared.set_index(keys='rank', inplace=True)
    
    return df1_compared, df2_compared

def getPrettyComparisonDataframe(df, title):
    s = df.style.applymap(colorComparisonField)
    s.set_caption(title)
    s.set_table_styles(table_styles)
    return s

In [4]:
conn = None
try:
    # read connection parameters
    paramsS17 = config.cfgAzureS17()
    paramsS03 = config.cfgAzureS03()

    paramsF17 = config.cfgAzureF17()
    paramsF03 = config.cfgAzureF03()
    
    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    connS17 = psycopg2.connect(**paramsS17)
    connS03 = psycopg2.connect(**paramsS03)
    
    connF17 = psycopg2.connect(**paramsF17)
    connF03 = psycopg2.connect(**paramsF03)

    # create a cursor
    curS17 = connS17.cursor()
    curS03 = connS03.cursor()
    
    curF17 = connF17.cursor()
    curF03 = connF03.cursor()

    # execute a statement
    print('PostgreSQL database version:')
    
    curS17.execute('SELECT version()')
    curS03.execute('SELECT version()')
    curF17.execute('SELECT version()')
    curF03.execute('SELECT version()')
    
    # display the PostgreSQL database server version
    db_version_curS17 = curS17.fetchone()
    db_version_curS03 = curS03.fetchone()
    db_version_curF17 = curF17.fetchone()
    db_version_curF03 = curF03.fetchone()
    
    print(db_version_curS17)
    print(db_version_curS03)
    print(db_version_curF17)
    print(db_version_curF03)

    # close the communication with the PostgreSQL
    curS17.close()
    curS03.close()
    curF17.close()
    curF03.close()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.6.9, compiled by Visual C++ build 1800, 64-bit',)
('PostgreSQL 9.6.9, compiled by Visual C++ build 1800, 64-bit',)
('PostgreSQL 9.6.9, compiled by Visual C++ build 1800, 64-bit',)
('PostgreSQL 9.6.9, compiled by Visual C++ build 1800, 64-bit',)


## Query URLs

### Calendar Week 03 - Sampled

In [5]:
#tweets03 = pd.read_sql_query("SELECT * FROM tweets_info;", conn, parse_dates=['created_at'] )
#tweets03['created_at'] = tweets['created_at'].dt.tz_localize("UTC").dt.tz_convert("Europe/Berlin")
tweetsS03 = pd.read_sql_query("SELECT tweet_id, url, title, text, domain, top_level_domain, response_code, img, hash FROM tweets_icards where has_icard is true;", connS03 )

print("Number of Tweets: %s" %len(tweetsS03))
tweetsS03.head()

Number of Tweets: 20568


,tweet_id,url,title,text,domain,top_level_domain,response_code,img,hash
0,952677513070809088,https://www.eyeem.com/p/117474810,Evening Winter Tower Rheinturm View Rhine Rhein | EyeEm,Photo by @oli170,https://www.eyeem.com/,eyeem.com,200.0,https://pbs.twimg.com/card_img/993108258385735680/6or4rjOA?format=jpg&name=600x314,fefedcf8fefdfefc
1,952677638879043586,None,None,None,None,None,NaN,None,None
2,952678171551457280,https://breaking-news-saarland.de/junge-frau-am-dillinger-bahnhof-sexuell-missbraucht/,Junge Frau am Dillinger Bahnhof sexuell missbraucht,None,https://breaking-news-saarland.de/,breaking-news-saarland.de,200.0,None,None
3,952679694087831552,None,None,None,None,None,NaN,None,None
4,952680591673167873,None,None,None,None,None,NaN,None,None


empty icards possible

### Calendar Week 17 - Sampled

In [6]:
tweetsS17 = pd.read_sql_query("SELECT tweet_id, url, title, text, domain, top_level_domain, response_code, img, hash FROM tweets_icards where has_icard is true;", connS17 )

print("Number of Tweets: %s" %len(tweetsS17))
tweetsS17.head()

Number of Tweets: 22033


,tweet_id,url,title,text,domain,top_level_domain,response_code,img,hash
0,990660022764998656,http://www.faz.net/aktuell/politik/ausland/oesterreich-diskutiert-ueber-die-kaltschnaeuzigen-blauhelme-15566647.html?utm_content=buffer316f8&utm_medium=social&utm_source=twitter.com&utm_campaign=GEPC%253Ds30,Blauhelme beobachten Morde: Dieses Video schockiert Österreich,Sie hätten die „Hunde“ warnen sollen: Ein kurzer Clip zeigt einen Hinterhalt am Golan. Zwei österreichische Blauhelme kommentieren das Blutbad. Ihre Kaltschnäuzigkeit wirft eine Frage auf.,http://www.faz.net/,faz.net,200.0,https://pbs.twimg.com/card_img/990643871183966210/A5Hmhyd9?format=jpg&name=600x314,f927fe1a26e1387c
1,988161845205913600,None,None,None,None,None,NaN,None,None
2,988152079490473990,None,None,None,None,None,NaN,None,None
3,988177036744241152,None,None,None,None,None,NaN,None,None
4,988177112250150912,None,None,None,None,None,NaN,None,None


### Calendar Week 03 - Filtered

In [7]:
tweetsF03 = pd.read_sql_query("SELECT tweet_id, url, title, text, domain, top_level_domain, response_code, img, hash FROM tweets_icards where has_icard is true;", connF03 )

print("Number of Tweets: %s" %len(tweetsF03))
tweetsF03.head()

Number of Tweets: 1254312


,tweet_id,url,title,text,domain,top_level_domain,response_code,img,hash
0,953517510300647424,https://www.golem.de/news/faser-wearables-oled-als-garn-fuer-leuchtende-stoffe-1801-132203.html,Faser-Wearables: OLED als Garn für leuchtende Stoffe - Golem.de,"Mit OLEDs, dünn wie ein Haar, könnten in Zukunft Stoffe gesponnen werden, die selbstleuchtend und sehr hell sind, aber wenig Energie benötigen. Damit könnten Displays auf",https://www.golem.de/,golem.de,200.0,https://pbs.twimg.com/card_img/1001857201324089346/ODL2V3dz?format=jpg&name=600x314,1b8e5972368bddcd
1,953517521424076800,https://www.heise.de/ix/meldung/Datenschutzgrundverordnung-Was-Admins-jetzt-wissen-muessen-3943750.html,Datenschutzgrundverordnung: Was Admins jetzt wissen müssen,"Ab Mai gelten die Regeln der neuen EU-Datenschutzgrundverordnung – und betreffen jedes Unternehmen. Ein Webinar erläutert Admins, Datenschutz- und Security-Verantwortlichen, wie sie sich vorbereiten...",https://www.heise.de/,heise.de,200.0,https://pbs.twimg.com/card_img/997927861540589569/AL4ENR-9?format=jpg&name=144x144_2,30ece1496df94961
2,953517520291471361,https://www.24pr.de/artikel/2459589?utm_source=dlvr.it&utm_medium=twitter,POL-KLE: Rheurdt – Einbruch in Pfarrheim / Zwei Blechdosen mit Spenden entwendet,"Rheurdt-Schaephuysen (ots) – In der Zeit zwischen Montag, 18.30 Uhr, und Dienstag (16. Januar 2018), 13.00 Uhr, schlugen unbekannte Täter an der Pastoratstraße eine Fensterscheibe an einem Pf…",https://www.24pr.de/,24pr.de,200.0,https://pbs.twimg.com/card_img/999200140580802560/2-BXhDU0?format=jpg&name=144x144_2,0000000000000000
3,953517522678173697,https://www.vn.at/lokal/vorarlberg/2018/01/16/ein-bruennele-und-seine-schiefe-optik.vn,Ein Brünnele und seine schiefe Optik,vandans Im Ortsteil Ganeu oberhalb von Vandans lebt es sich gut.,https://www.vn.at/,vn.at,200.0,None,None
4,953517523089182720,https://www.heise.de/ix/meldung/Datenschutzgrundverordnung-Was-Admins-jetzt-wissen-muessen-3943750.html,Datenschutzgrundverordnung: Was Admins jetzt wissen müssen,"Ab Mai gelten die Regeln der neuen EU-Datenschutzgrundverordnung – und betreffen jedes Unternehmen. Ein Webinar erläutert Admins, Datenschutz- und Security-Verantwortlichen, wie sie sich vorbereiten...",https://www.heise.de/,heise.de,200.0,https://pbs.twimg.com/card_img/997927861540589569/AL4ENR-9?format=jpg&name=144x144_2,30ece1496df94961


### Calendar Week 17 - Filtered

In [8]:
tweetsF17 = pd.read_sql_query("SELECT tweet_id, url, title, text, domain, top_level_domain, response_code, img, hash FROM tweets_icards where has_icard is true;", connF17 )

print("Number of Tweets: %s" %len(tweetsF17))
tweetsF17.head()

Number of Tweets: 1312874


,tweet_id,url,title,text,domain,top_level_domain,response_code,img,hash
0,989827499159883783,https://www.youtube.com/watch?v=Q1TXk7ZdiQE,"Zlatan Ibrahimović on Playing for LA Galaxy, His Nicknames & The...","Zlatan talks about coming to LA to play soccer, his favorite nickname that the fans have given him, scoring his first goal for the LA Galaxy, his very high c...",https://www.youtube.com/,youtube.com,200.0,https://pbs.twimg.com/card_img/999168427041480704/IYY3nwhZ?format=jpg&name=280x280,663272f29292c683
1,989827499856224258,http://www.faz.net/aktuell/politik/die-atomwaffen-wird-kim-letztlich-nicht-hergeben-15563301.html?GEPC=s30&utm_content=buffere75d2&utm_medium=social&utm_source=twitter.com&utm_campaign=GEPC%253Ds30,Vollständige Abrüstung?: Die Atomwaffen wird Kim letztlich nicht hergeben,"Nord- und Südkorea wollen offiziell den Kriegszustand beenden und atomar abrüsten, so zumindest die Absichtserklärung. Welche Interessen hinter dem Treffen stecken, erklärt unser Redakteur Martin...",http://www.faz.net/,faz.net,200.0,https://pbs.twimg.com/card_img/997718942792732674/kFv-hhw9?format=jpg&name=600x314,c624323333736af4
2,989827500103618561,https://www.handelszeitung.ch/unternehmen/nestle-spurt-umsatzverlust-wegen-streit?utm_source=Handelszeitung+Newsletter&utm_campaign=58999ff943-EMAIL_CAMPAIGN_LUNCH_2017_11_02&utm_medium=email&utm_term=0_c1505081ea-58999ff943-98565329,Nestlé spürt Umsatzverlust wegen Streit - Handelszeitung,Nestlé verzeichnet in Deutschland wegen des Einkaufsstreits mit Edeka Umsatzverluste. Wie viel sei aber nicht messbar.,https://www.handelszeitung.ch/,handelszeitung.ch,200.0,https://pbs.twimg.com/card_img/999783799264333824/GkK7wx4L?format=jpg&name=600x314,5f591b7770707931
3,989827500892196864,https://www.youtube.com/watch?v=KRACRRuUxxY,Romanian Police VS. Door | EPIC FAIL,13 min battle with police lose and door win,https://www.youtube.com/,youtube.com,200.0,https://pbs.twimg.com/card_img/999076540158693376/_hTdg8PH?format=jpg&name=280x280,433373179f9b9b87
4,989827500611162112,None,None,None,None,None,NaN,None,None


## Top Level Domains

### Calendar Week 03 - TLDs - Sampled x Filtered

In [10]:
attribute = 'top_level_domain'
column_name = 'Top Level Domain'
    
df_sampled = generateRankingDataframe(tweetsS03[attribute].value_counts(), column_name)
df_filtered = generateRankingDataframe(tweetsF03[attribute].value_counts(), column_name)

df_sampled_comparison, df_filtered_comparison = generateComparisonDataframes(df_sampled, df_filtered, column_name, 25)

In [12]:
s = getPrettyComparisonDataframe(df_sampled_comparison, "CW 17 - Top Level Domains - Sampled Data (compared to Filtered Data)")
s

,Top Level Domain,difference (rank / percentage / diff),percentage,value
rank,,,,
1,youtube.com,= (1. / 15.636% / -1.611%),14.0258,2451
2,welt.de,= (2. / 3.022% / 0.280%),3.30186,577
3,spiegel.de,= (3. / 2.331% / 0.015%),2.34621,410
4,focus.de,= (4. / 2.317% / -0.171%),2.14592,375
5,bild.de,v (6. / 1.860% / 0.183%),2.04292,357
6,faz.net,v (7. / 1.783% / 0.008%),1.79113,313
7,zeit.de,v (9. / 1.400% / 0.202%),1.60229,280
8,sueddeutsche.de,v (10. / 1.317% / 0.245%),1.56223,273
9,presseportal.de,v (11. / 1.311% / 0.131%),1.44206,252


In [13]:
s = getPrettyComparisonDataframe(df_filtered_comparison, "CW 03 - Top Level Domains - Filtered Data (compared to Sampled Data)")
s

,Top Level Domain,difference (rank / percentage / diff),percentage,value
rank,,,,
1,youtube.com,= (1. / 14.026% / 1.611%),15.6365,178108
2,welt.de,= (2. / 3.302% / -0.280%),3.02215,34424
3,spiegel.de,= (3. / 2.346% / -0.015%),2.33114,26553
4,focus.de,= (4. / 2.146% / 0.171%),2.31701,26392
5,google.com,v (14. / 0.847% / 1.049%),1.89561,21592
6,bild.de,^ (5. / 2.043% / -0.183%),1.85979,21184
7,faz.net,^ (6. / 1.791% / -0.008%),1.78314,20311
8,twitch.tv,v (10. / 1.276% / 0.155%),1.43084,16298
9,zeit.de,^ (7. / 1.602% / -0.202%),1.39993,15946


### Calendar Week 17 - TLDs - Sampled x Filtered

In [62]:
attribute = 'top_level_domain'
column_name = 'Top Level Domain'
    
df_sampled = generateRankingDataframe(tweetsS17[attribute].value_counts(), column_name)
df_filtered = generateRankingDataframe(tweetsF17[attribute].value_counts(), column_name)

df_sampled_comparison, df_filtered_comparison = generateComparisonDataframes(df_sampled, df_filtered, column_name, 25)

In [63]:
s = getPrettyComparisonDataframe(df_sampled_comparison, "CW 17 - Top Level Domains - Sampled Data (compared to Filtered Data)")
s

,Top Level Domain,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,youtube.com,2576,14.0542,= (1. / 15.294% / -1.240%)
2,welt.de,501,2.73337,= (2. / 2.579% / 0.155%)
3,spiegel.de,389,2.12232,= (3. / 2.172% / -0.050%)
4,focus.de,387,2.11141,= (4. / 1.961% / 0.151%)
5,bild.de,347,1.89317,= (5. / 1.908% / -0.014%)
6,faz.net,306,1.66949,= (6. / 1.658% / 0.012%)
7,zeit.de,284,1.54946,= (7. / 1.459% / 0.090%)
8,sueddeutsche.de,246,1.34214,v (9. / 1.360% / -0.018%)
9,twitch.tv,241,1.31486,^ (8. / 1.442% / -0.127%)


In [61]:
s = getPrettyComparisonDataframe(df_filtered_comparison, "CW 03 - Top Level Domains - Filtered Data (compared to Sampled Data)")
s

,Top Level Domain,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,youtube.com,180978,15.2939,= (1. / 14.054% / 1.240%)
2,welt.de,30514,2.57865,= (2. / 2.733% / -0.155%)
3,spiegel.de,25706,2.17234,= (3. / 2.122% / 0.050%)
4,focus.de,23200,1.96057,= (4. / 2.111% / -0.151%)
5,bild.de,22574,1.90767,= (5. / 1.893% / 0.014%)
6,faz.net,19617,1.65778,= (6. / 1.669% / -0.012%)
7,zeit.de,17268,1.45927,= (7. / 1.549% / -0.090%)
8,twitch.tv,17063,1.44195,v (9. / 1.315% / 0.127%)
9,sueddeutsche.de,16089,1.35964,^ (8. / 1.342% / 0.018%)


## URLs

### Calendar Week 03 - URLs - Sampled x Filtered

In [64]:
attribute = 'url'
column_name = 'url'
    
df_sampled = generateRankingDataframe(tweetsS03[attribute].value_counts(), column_name)
df_filtered = generateRankingDataframe(tweetsF03[attribute].value_counts(), column_name)

df_sampled_comparison, df_filtered_comparison = generateComparisonDataframes(df_sampled, df_filtered, column_name, 25)

In [66]:
s = getPrettyComparisonDataframe(df_sampled_comparison, "CW 03 - URLs - Sampled Data (compared to Filtered Data)")
s

,url,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,51,0.291845,v (3. / 0.172% / 0.120%)
2,https://pi2.17bullets.com/tw_post.php?messageId=achievement&values=&locale=en_US,44,0.251788,^ (1. / 0.411% / -0.159%)
3,http://www.the-sz.com/products/vbbinfo/?f=3,34,0.194564,^ (2. / 0.261% / -0.066%)
4,https://www.radionomy.com/en/radio/erika1,27,0.154506,v (5. / 0.142% / 0.012%)
5,http://www.achgut.com/artikel/der_groko_betrug_mit_dem_asylrecht,25,0.143062,v (7. / 0.130% / 0.013%)
6,http://www.radio-jodlerwirt.de,22,0.125894,^ (4. / 0.143% / -0.017%)
7,http://www.spiegel.de/politik/deutschland/fluechtlinge-eu-fluechtlingsplaene-alarmieren-bundesregierung-a-1187500.html,21,0.120172,v (17. / 0.082% / 0.038%)
8,https://www.timesofisrael.com/stabbing-victim-pulls-knife-out-of-own-neck-kills-terrorist/,19,0.108727,v (9. / 0.118% / -0.009%)
9,https://www.amazon.com/Shally-Vintage-Bracelet-Tassel-Bohemia/dp/B0753D9LQ4/ref=sr_1_2?s=apparel&ie=UTF8&qid=1514463817&sr=1-2&nodeID=7147440011&psd=1&keywords=coin+bracelets+for+women,17,0.0972818,-


In [67]:
s = getPrettyComparisonDataframe(df_filtered_comparison, "CW 03 - URLs - Filtered Data (compared to Sampled Data)")
s

,url,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,https://pi2.17bullets.com/tw_post.php?messageId=achievement&values=&locale=en_US,4680,0.410864,v (2. / 0.252% / 0.159%)
2,http://www.the-sz.com/products/vbbinfo/?f=3,2973,0.261004,v (3. / 0.195% / 0.066%)
3,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,1959,0.171983,^ (1. / 0.292% / -0.120%)
4,http://www.radio-jodlerwirt.de,1628,0.142924,v (6. / 0.126% / 0.017%)
5,https://www.radionomy.com/en/radio/erika1,1621,0.14231,^ (4. / 0.155% / -0.012%)
6,https://www.amazon.com/Shally-Stylish-Artificial-Necklace-Valentines/dp/B0753DNRJL/ref=sr_1_5?s=apparel&ie=UTF8&qid=1514462386&sr=1-5&nodeID=7141123011&psd=1&keywords=valentine+day+gifts+for+her,1549,0.135989,v (50. / 0.052% / 0.084%)
7,http://www.achgut.com/artikel/der_groko_betrug_mit_dem_asylrecht,1479,0.129843,^ (5. / 0.143% / -0.013%)
8,https://sec.help.ch,1466,0.128702,-
9,https://www.timesofisrael.com/stabbing-victim-pulls-knife-out-of-own-neck-kills-terrorist/,1343,0.117904,^ (8. / 0.109% / 0.009%)


### Calendar Week 17 - URLs - Sampled x Filtered

In [68]:
attribute = 'url'
column_name = 'url'
    
df_sampled = generateRankingDataframe(tweetsS17[attribute].value_counts(), column_name)
df_filtered = generateRankingDataframe(tweetsF17[attribute].value_counts(), column_name)

df_sampled_comparison, df_filtered_comparison = generateComparisonDataframes(df_sampled, df_filtered, column_name, 25)

In [69]:
s = getPrettyComparisonDataframe(df_sampled_comparison, "CW 17 - URLs - Sampled Data (compared to Filtered Data)")
s

,url,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,https://pi2.17bullets.com/tw_post.php?messageId=achievement&values=&locale=en_US,56,0.305527,= (1. / 0.419% / -0.114%)
2,http://www.achgut.com/artikel/die_enthauptung_der_hamburger_justiz,44,0.240057,v (4. / 0.228% / 0.012%)
3,https://www.youtube.com/watch?v=F-eMt3SrfFU&feature=youtu.be,34,0.185498,v (1003. / 0.007% / 0.179%)
4,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,33,0.180043,v (7. / 0.163% / 0.017%)
5,https://www.focus.de/politik/deutschland/lage-zu-bedrohlich-demonstration-gegen-antisemitismus-in-berlin-nach-15-minuten-abgebrochen_id_8829384.html,29,0.158219,v (13. / 0.104% / 0.055%)
6,http://www.the-sz.com/products/vbbinfo/?f=3,27,0.147308,^ (3. / 0.231% / -0.084%)
7,http://www.linkedin.com/,24,0.13094,^ (6. / 0.164% / -0.034%)
8,http://www.gooni168.tv/sellpanties/item/haushalts-sklave-in/,22,0.120028,v (24112. / 0.001% / 0.120%)
9,https://www.youtube.com/watch?v=yiWCGP4iKf4,21,0.114573,v (102656. / 0.000% / 0.114%)


In [70]:
s = getPrettyComparisonDataframe(df_filtered_comparison, "CW 17 - URLs - Filtered Data (compared to Sampled Data)")
s

,url,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,https://pi2.17bullets.com/tw_post.php?messageId=achievement&values=&locale=en_US,4964,0.419488,= (1. / 0.306% / 0.114%)
2,https://sec.help.ch,2978,0.251659,-
3,http://www.the-sz.com/products/vbbinfo/?f=3,2739,0.231462,v (6. / 0.147% / 0.084%)
4,http://www.achgut.com/artikel/die_enthauptung_der_hamburger_justiz,2697,0.227913,^ (2. / 0.240% / -0.012%)
5,https://www.radionomy.com/en/radio/erika1,2172,0.183547,v (12. / 0.109% / 0.074%)
6,http://www.linkedin.com/,1946,0.164449,v (7. / 0.131% / 0.034%)
7,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,1932,0.163266,^ (4. / 0.180% / -0.017%)
8,http://www.radio-jodlerwirt.de,1785,0.150843,v (21. / 0.082% / 0.069%)
9,https://www.welt.de/debatte/kommentare/article175695478/Straftaten-Statistik-Die-Wirklichkeit-hinter-den-neuen-Zahlen-zur-Kriminalitaet.html,1510,0.127604,v (16. / 0.098% / 0.029%)


## Hashes

### Calendar Week 03 - Hashes - Sampled x Filtered

In [71]:
attribute = 'hash'
column_name = 'hash'
    
df_sampled = generateRankingDataframe(tweetsS03[attribute].value_counts(), column_name)
df_filtered = generateRankingDataframe(tweetsF03[attribute].value_counts(), column_name)

df_sampled_comparison, df_filtered_comparison = generateComparisonDataframes(df_sampled, df_filtered, column_name, 25)

In [73]:
s = getPrettyComparisonDataframe(df_sampled_comparison, "CW 03 - Hash - Sampled Data (compared to Filtered Data)")
s

,hash,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,0000000000000000,1244,11.0333,= (1. / 12.336% / -1.302%)
2,96a4a29696a6b6b6,55,0.487805,v (6. / 0.322% / 0.165%)
3,734545135755654d,52,0.461197,^ (2. / 0.722% / -0.260%)
4,0d2f67a6b3731393,48,0.425721,-
5,0c8e136b0b3fc661,41,0.363636,^ (3. / 0.431% / -0.068%)
6,1004312ab2320c20,39,0.345898,v (8. / 0.224% / 0.122%)
7,3323332b23070fb3,34,0.301552,^ (4. / 0.390% / -0.088%)
8,254d8c0f0f0e4d5a,29,0.257206,v (9. / 0.222% / 0.035%)
9,26c9c9e4147a6916,28,0.248337,^ (5. / 0.363% / -0.115%)


In [82]:
tweetsS03[tweetsS03['hash'] == '0d2f67a6b3731393']

,tweet_id,url,title,text,domain,top_level_domain,response_code,img,hash
3009,954649980970684416,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
4250,952761789195923456,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
4726,952838276553019392,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
5459,952917233973506048,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
5460,952918245144715264,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
5488,952881111654420480,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
5691,952939778676350976,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
5794,952947248698150912,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutsch

In [74]:
s = getPrettyComparisonDataframe(df_filtered_comparison, "CW 03 - Hash - Filtered Data (compared to Sampled Data)")
s

,hash,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,0000000000000000,78499,12.3357,= (1. / 11.033% / 1.302%)
2,734545135755654d,4592,0.721611,v (3. / 0.461% / 0.260%)
3,0c8e136b0b3fc661,2745,0.431364,v (5. / 0.364% / 0.068%)
4,3323332b23070fb3,2482,0.390034,v (7. / 0.302% / 0.088%)
5,26c9c9e4147a6916,2309,0.362848,v (9. / 0.248% / 0.115%)
6,96a4a29696a6b6b6,2052,0.322462,^ (2. / 0.488% / -0.165%)
7,46072764f0e4ec7b,1458,0.229118,v (12. / 0.204% / 0.025%)
8,1004312ab2320c20,1423,0.223618,^ (6. / 0.346% / -0.122%)
9,254d8c0f0f0e4d5a,1414,0.222203,^ (8. / 0.257% / -0.035%)


In [81]:
tweetsF03[tweetsF03['hash'] == 'b771607240588bcc']

,tweet_id,url,title,text,domain,top_level_domain,response_code,img,hash
523,954686868674764800,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
824,954308029742239744,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
1535,953221824011218945,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
1868,953942835224633344,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
3728,954687625230802944,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
3957,954308787657232384,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
4309,953222580709875713,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
5090,953943591851888640,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
6504,954688382063521798,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat

### Calendar Week 17 - Hashes - Sampled x Filtered

In [83]:
attribute = 'hash'
column_name = 'hash'
    
df_sampled = generateRankingDataframe(tweetsS17[attribute].value_counts(), column_name)
df_filtered = generateRankingDataframe(tweetsF17[attribute].value_counts(), column_name)

df_sampled_comparison, df_filtered_comparison = generateComparisonDataframes(df_sampled, df_filtered, column_name, 25)

In [84]:
s = getPrettyComparisonDataframe(df_sampled_comparison, "CW 03 - Hash - Sampled Data (compared to Filtered Data)")
s

,hash,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,0000000000000000,602,3.79955,= (1. / 7.337% / -3.537%)
2,734545135755654d,63,0.397627,= (2. / 0.630% / -0.233%)
3,26c9c9e4147a6916,44,0.277708,= (3. / 0.475% / -0.197%)
4,72e0c9c360614adb,44,0.277708,v (6. / 0.298% / -0.020%)
5,78b2b64418292b73,43,0.271396,v (23. / 0.135% / 0.137%)
6,e06cefceaeea7b06,40,0.252461,v (1038. / 0.009% / 0.244%)
7,434b486474f8b6fe,34,0.214592,v (8. / 0.253% / -0.038%)
8,f453696d696b4727,31,0.195658,v (25. / 0.133% / 0.063%)
9,254d8c0f0f0e4d5a,30,0.189346,^ (7. / 0.270% / -0.081%)


In [82]:
tweetsS17[tweetsS17['hash'] == '0d2f67a6b3731393']

,tweet_id,url,title,text,domain,top_level_domain,response_code,img,hash
3009,954649980970684416,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
4250,952761789195923456,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
4726,952838276553019392,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
5459,952917233973506048,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
5460,952918245144715264,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
5488,952881111654420480,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
5691,952939778676350976,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutschlandfunk.de,200.0,https://pbs.twimg.com/card_img/992455900471480321/e1x2CIJC?format=jpg&name=144x144_2,0d2f67a6b3731393
5794,952947248698150912,http://www.deutschlandfunk.de/dlf24-startseite.1441.de.html,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,An der Grenze zwischen dem Gazastreifen und Israel hat es erneut Auseinandersetzungen zwischen israelischen Soldaten und Palästinensern gegeben. Nach Angaben des Gesundheitsministeriums in Gaza...,http://www.deutschlandfunk.de/,deutsch

In [86]:
s = getPrettyComparisonDataframe(df_filtered_comparison, "CW 17 - Hash - Filtered Data (compared to Sampled Data)")
s

,hash,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,0000000000000000,55873,7.337,= (1. / 3.800% / 3.537%)
2,734545135755654d,4799,0.630184,= (2. / 0.398% / 0.233%)
3,26c9c9e4147a6916,3616,0.474837,= (3. / 0.278% / 0.197%)
4,0c13212531232704,3384,0.444372,v (803. / 0.019% / 0.425%)
5,3323332b23070fb3,2313,0.303733,v (13. / 0.170% / 0.133%)
6,72e0c9c360614adb,2268,0.297824,^ (4. / 0.278% / 0.020%)
7,254d8c0f0f0e4d5a,2059,0.270379,v (9. / 0.189% / 0.081%)
8,434b486474f8b6fe,1923,0.25252,^ (7. / 0.215% / 0.038%)
9,0c8e136b0b3fc661,1743,0.228883,v (17. / 0.151% / 0.077%)


In [81]:
tweetsF03[tweetsF03['hash'] == 'b771607240588bcc']

,tweet_id,url,title,text,domain,top_level_domain,response_code,img,hash
523,954686868674764800,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
824,954308029742239744,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
1535,953221824011218945,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
1868,953942835224633344,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
3728,954687625230802944,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
3957,954308787657232384,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
4309,953222580709875713,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
5090,953943591851888640,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat sich – trotz gesunkener Preise – um rund sieben Prozent verteuert. Kosten für Gas höher als 2016,http://www.wpall.de/,wpall.de,200.0,https://pbs.twimg.com/card_img/1001631178338848770/ZFnRZ2br?format=jpg&name=144x144_2,b771607240588bcc
6504,954688382063521798,http://www.wpall.de/beginn-der-heizperiode-kosten-fuer-gas-und-heizoel-hoeher-als-2016/,Beginn der Heizperiode: Kosten für Gas und Heizöl höher als 2016,Kosten für Gas höher als 2016! Heizen mit Gas hat

## Titles

### Calendar Week 03 - Titles - Sampled x Filtered

In [91]:
attribute = 'title'
column_name = 'title'
    
df_sampled = generateRankingDataframe(tweetsS03[attribute].value_counts(), column_name)
df_filtered = generateRankingDataframe(tweetsF03[attribute].value_counts(), column_name)

df_sampled_comparison, df_filtered_comparison = generateComparisonDataframes(df_sampled, df_filtered, column_name, 25)

In [92]:
s = getPrettyComparisonDataframe(df_sampled_comparison, "CW 03 - Titles - Sampled Data (compared to Filtered Data)")
s

,title,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,Paradise Island 2,52,0.297551,= (1. / 0.489% / -0.192%)
2,Proteste am Gazastreifen Palästinenser sprechen von 350 Verletzten,48,0.274662,-
3,VBB Info,48,0.274662,^ (2. / 0.456% / -0.181%)
4,Petition unterschreiben,39,0.223163,^ (3. / 0.243% / -0.020%)
5,Radionomy,29,0.165942,= (5. / 0.156% / 0.010%)
6,Der GroKo-Betrug mit dem Asylrecht,25,0.143053,v (9. / 0.130% / 0.013%)
7,Schiff,24,0.137331,^ (4. / 0.174% / -0.037%)
8,Der Volksmusiksender,23,0.131609,^ (6. / 0.155% / -0.024%)
9,Vorschläge des Europaparlaments: Bundesregierung alarmiert über EU-Flüchtlingspläne - SPIEGEL...,21,0.120165,v (20. / 0.082% / 0.038%)


In [93]:
s = getPrettyComparisonDataframe(df_filtered_comparison, "CW 03 - Titles - Filtered Data (compared to Sampled Data)")
s

,title,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,Paradise Island 2,5572,0.489152,= (1. / 0.298% / 0.192%)
2,VBB Info,5194,0.455968,v (3. / 0.275% / 0.181%)
3,Petition unterschreiben,2772,0.243347,v (4. / 0.223% / 0.020%)
4,Schiff,1987,0.174434,v (7. / 0.137% / 0.037%)
5,Radionomy,1781,0.156349,= (5. / 0.166% / -0.010%)
6,Der Volksmusiksender,1770,0.155384,v (8. / 0.132% / 0.024%)
7,Landeshauptstadt Dresden fördert merkwürdigen Kurs - Burka-Anprobe in der Volkshochschule,1574,0.138177,v (13. / 0.109% / 0.029%)
8,Shally Women's Stylish Artificial Gem Love Heart Shape Pendant Chain Necklace Valentines Gift,1549,0.135983,v (73. / 0.051% / 0.084%)
9,Der GroKo-Betrug mit dem Asylrecht,1479,0.129838,^ (6. / 0.143% / -0.013%)


### Calendar Week 17 - Titles - Sampled x Filtered

In [94]:
attribute = 'title'
column_name = 'title'
    
df_sampled = generateRankingDataframe(tweetsS17[attribute].value_counts(), column_name)
df_filtered = generateRankingDataframe(tweetsF17[attribute].value_counts(), column_name)

df_sampled_comparison, df_filtered_comparison = generateComparisonDataframes(df_sampled, df_filtered, column_name, 25)

In [95]:
s = getPrettyComparisonDataframe(df_sampled_comparison, "CW 17 - Titles - Sampled Data (compared to Filtered Data)")
s

,title,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,Paradise Island 2,63,0.343718,= (1. / 0.492% / -0.148%)
2,VBB Info,56,0.305527,= (2. / 0.435% / -0.130%)
3,Die Enthauptung der Hamburger Justiz,44,0.240057,v (5. / 0.232% / 0.008%)
4,Dunkirk - Trailer 1 [HD],40,0.218233,v (955. / 0.007% / 0.211%)
5,Petition unterschreiben,37,0.201866,^ (4. / 0.261% / -0.059%)
6,Schiff,32,0.174587,^ (3. / 0.281% / -0.106%)
7,An der Uni Bielefeld kann man nun masturbieren lernen - WELT,31,0.169131,v (16. / 0.106% / 0.063%)
8,Radionomy,30,0.163675,^ (6. / 0.204% / -0.040%)
9,Lage zu bedrohlich: Berliner Antisemitismus-Demo nach 15 Minuten abgebrochen,29,0.158219,v (14. / 0.108% / 0.050%)


In [96]:
s = getPrettyComparisonDataframe(df_filtered_comparison, "CW 03 - Titles - Filtered Data (compared to Sampled Data)")
s

,title,value,percentage,difference (rank / percentage / diff)
rank,,,,
1,Paradise Island 2,5821,0.49188,= (1. / 0.344% / 0.148%)
2,VBB Info,5150,0.43518,= (2. / 0.306% / 0.130%)
3,Schiff,3323,0.280797,v (6. / 0.175% / 0.106%)
4,Petition unterschreiben,3083,0.260517,v (5. / 0.202% / 0.059%)
5,Die Enthauptung der Hamburger Justiz,2748,0.232209,^ (3. / 0.240% / -0.008%)
6,Radionomy,2409,0.203563,v (8. / 0.164% / 0.040%)
7,Der Volksmusiksender,1979,0.167227,v (31. / 0.082% / 0.085%)
8,Straftaten-Statistik: Die Wirklichkeit hinter den neuen Zahlen zur Kriminalität - WELT,1866,0.157679,v (18. / 0.109% / 0.049%)
9,"Wenige Stunden, um die Bienen zu retten",1679,0.141877,v (10. / 0.153% / -0.011%)
